In [ ]:
import openai
import sqlite3
import pandas as pd
from langchain_openai import ChatOpenAI
import os
from sqlalchemy import create_engine
import numpy as np
import sqlalchemy
from openai import OpenAI
from llama_index.core import SQLDatabase  # ✅ Correct Import
from llama_index.llms.openai import OpenAI
from sqlalchemy import text
from llama_index.core.query_engine import NLSQLTableQueryEngine






os.environ['OPENAI_API_KEY'] = ""


In [ ]:
# import nba_on_court as noc
# noc.load_nba_data(seasons=2022, data=('nbastats', 'pbpstats'), seasontype='po', untar=True)


In [ ]:
nbastats = pd.read_csv('nbastats_po_2022.csv')
pbpstats = pd.read_csv('pbpstats_po_2022.csv')

In [ ]:
# Define database name
db_path = "nba_database.sqlite"

# Connect to SQLite
conn = sqlite3.connect(db_path)

# Read CSV files into Pandas DataFrames
nbastats = pd.read_csv('nbastats_po_2022.csv')
pbpstats = pd.read_csv('pbpstats_po_2022.csv')

# Store data in SQLite tables
nbastats.to_sql("nba_stats_2022", conn, if_exists="replace", index=False)
pbpstats.to_sql("pbp_stats_2022", conn, if_exists="replace", index=False)

# Verify table creation
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM nba_stats_2022;")
nbastats_count = cursor.fetchone()[0]

cursor.execute("SELECT COUNT(*) FROM pbp_stats_2022;")
pbpstats_count = cursor.fetchone()[0]

print(f"✅ Successfully inserted {nbastats_count} rows into nba_stats_2022.")
print(f"✅ Successfully inserted {pbpstats_count} rows into pbp_stats_2022.")

# Close connection
conn.close()


In [ ]:
client = OpenAI(organization='org-ixwhtUweym8awcsuUS7A1EeE')

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello, how are you?"}]
)



In [ ]:
db_path = "nba_database.sqlite"


In [ ]:
engine = create_engine(f"sqlite:///{db_path}")
sql_db = SQLDatabase(engine)



In [ ]:
with engine.connect() as con:
    rows = con.execute(text("SELECT PLAYER1_NAME from nba_stats_2022"))
    for row in rows:
        print(row)

In [ ]:


llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo")


In [ ]:
import sqlite3  # Use MySQL connector if needed
import random

# Database connection (change for MySQL or other databases)


def get_random_row():
    try:
        # Connect to the database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Query to fetch a random row (for SQLite)
        cursor.execute("SELECT * FROM pbp_stats_2022 ORDER BY RANDOM() LIMIT 1;")

        # Fetch the result
        row = cursor.fetchone()
        column_names = [desc[0] for desc in cursor.description]

        # Close the connection
        conn.close()

        return dict(zip(column_names, row)) if row else "No data found."


    except Exception as e:
        return f"Error: {e}"



In [ ]:
random_row = get_random_row()
print(str(random_row))


In [ ]:
query_str = f"""
Based on the following live play-by-play event, generate an insightful real-time statistical fact about any player involved.
Ensure the fact is based only on data **before this event** and sounds like it is being delivered by a commentator or analyst.

Include relevant game context such as:
- The **current score, quarter, and time** remaining when the event occurred.
- The player's **performance in this game so far** (e.g., points, assists, rebounds).
- Comparison to:
  1. Their season or career average in the relevant stat category before this game.
  2. Their team's or the league's average for the season before this game.
  3. Their past performances in similar situations before this game.

Make sure the fact is **numerical, relevant, and can be confirmed by the dataset**.
Do not reference any data from the future or speculate on upcoming plays.

Live Play-By-Play Event:
{random_row}
"""




In [ ]:
query_str

In [ ]:
import llama_index.core

llama_index.core.set_global_handler("simple")

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_db, tables=["pbp_stats_2022"], llm=llm
)


response = query_engine.query(query_str)

In [ ]:
response.metadata["result"]

In [ ]:
response.response